In [13]:
import numpy as np
import pandas as pd
import seaborn as sn
from copy import copy
import matplotlib.pyplot as plt
from scipy.linalg import hilbert, solve
from numpy import matrix

In [19]:
def spin_method(a):  # Разложение методом вращений
    def t(i, j, a):  # Строим матрицу вращения
        x = a[i, i]
        y = a[i, j]
        root = np.sqrt(x*x + y*y)
        cos = x/root
        sin = -y/root
        m = matrix(np.eye(a.shape[0], a.shape[0]))
        m[i, i] = cos
        m[i, j] = -sin
        m[j, j] = cos
        m[j, i] = sin
        return m
    
    r = copy(a)
    n = a.shape[0]
    q = np.eye(n)
    for i in range(n-1):
        for j in range(i+1, n):
            t_temp = t(i, j, a)
            r = t_temp * r
            q = q * t_temp.T
    return q, r

In [3]:
def norm(matrix):  # Спектральная норма матрицы
    return np.linalg.norm(x=matrix, ord=2)


def cond(matrix):  # Спектральный критерий обусловленности
    return norm(matrix) * norm(matrix.getI())


def sqrt_decomposition(a):  # Метод квадратого корня
    n = a.shape[0]
    l = matrix(np.zeros((n, n)))
    for i in range(n):
        for j in range(i):
            l[i, j] = (a[i, j] - sum([l[i, k]*l[j, k] for k in range(j)])) / l[j, j]
        l[i, i] = np.sqrt(a[i, i]-sum([l[i, k]*l[i, k] for k in range(i)]))
    return l

def solve_root(a, b):  # Решение системы через разложение методом квадратного корня
    l = sqrt_decomposition(a)
    y = solve(l, b)
    x = solve(l.T, y)
    return x

def solve_spin(a, b):  # Решение системы через разложение методом вращений
    q, r = spin_method(a)
    y = solve(q, b)
    x = solve(r, y)
    return x    


def regularization(a, b):
    n = a.shape[0]
    conds = [cond(a)]
    errors = [0]
    alphas = [0]
    x = solve_spin(a, b)
    for alpha in range(-12, 0):
        a_alpha = a + (10**alpha)*matrix(np.eye(n))
        conds.append(cond(a_alpha))
        x_alpha = solve_spin(a_alpha, b)
        errors.append(norm(x_alpha-x))
        alphas.append(10**alpha)
    res = list(zip(alphas, conds, errors))
    return res

In [5]:
a1 = matrix(hilbert(5))
a2 = matrix(hilbert(10))
x1 = np.ones((5, 1))
x2 = np.ones((10, 1))
b1 = a1*x1
b2 = a2*x2

In [6]:
print("Сравнение норм разности для матриц Гильберта")
print("Порядок 5")
print("Метод корня: {}\nМетод вращений: {}".format(norm(solve_root(a1, b1) - x1), norm(solve_spin(a1, b1) - x1)))
print("Порядок 10")
print("Метод корня: {}\nМетод вращений: {}".format(norm(solve_root(a2, b2) - x2), norm(solve_spin(a2, b2) - x2)))

Сравнение норм разности для матриц Гильберта
Порядок 5
Метод корня: 4.891166397860348e-12
Метод вращений: 1.9414159060339524e-11
Порядок 10
Метод корня: 0.00011220285980174058
Метод вращений: 0.0015320813003343338


In [7]:
def main(a, b):
    res = regularization(a, b)
    print('A =\n', a, '\nb =\n', b)
    print('\n\n{:<8}{:<12}{:<7}'.format('Alpha', 'Cond(A)', 'Модуль невязки'))
    for item in res:
        print('{:<8} {:<12.0f} {:<7}'.format(item[0], item[1], item[2]))

In [8]:
def test(a, alpha):
    n = a.shape[0]
    x0 = generatex0(n)
    print('Значение фиксированного alpha:', alpha)
    print('Норма x0(решения):', norm(x0))
    b = a * x0
    res = []
    res.append(norm(solve_spin(a+0.1*alpha*matrix(np.eye(n)), b)-x0))
    res.append(norm(solve_spin(a+alpha*matrix(np.eye(n)), b)-x0))
    res.append(norm(solve_spin(a+10*alpha*matrix(np.eye(n)), b)-x0))
    print("Нормы разности (|x-x_alpha|):")
    print("0.1alpha", res[0], '\nalpha', res[1], '\n10alpha', res[2])

In [9]:
main(a1, b1)
main(a2, b2)

A =
 [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]] 
b =
 [[2.28333333]
 [1.45      ]
 [1.09285714]
 [0.88452381]
 [0.74563492]]


Alpha   Cond(A)     Модуль невязки
0        476607       0      
1e-12    476607       1.3753435909026073e-09
1e-11    476606       1.4370254842637523e-08
1e-10    476593       1.4361958569377103e-07
1e-09    476462       1.4357419117582609e-06
1e-08    475162       1.4318729308917871e-05
1e-07    462539       0.0001394205855572008
1e-06    365457       0.001104924477275212
1e-05    117931       0.003792613351409736
0.0001   15173        0.011673447070005581
0.001    1563         0.038779276216364275
0.01     158          0.1300612420360287
0.1      17           0.4044243648415419
A =
 [[1.         0.5        0.333333